## Comparison of deviations of differences - and of the actual embeddings

In [1]:
import pandas as pd
import json

In [2]:
tf27 = pd.read_csv("comparison-tf27.csv")
tf26 = pd.read_csv("comparison-tf26.csv")

In [3]:
import numpy as np

In [4]:
for df in [tf26, tf27]:
    df['model_weights'].fillna('None', inplace=True)
    for type_ in ['sequence', 'sentence'] : 
        for what in ['expected', 'actual' ] :
            df[f'{type_}_{what}'] = df[f'{type_}_{what}'].apply(lambda x : json.loads(x))

In [5]:
id_cols = ['model_name', 'model_weights', 'text']
suffixes = ['_27', '_26']

### Compare the deviations from expected

In [6]:
both = pd.merge(tf27, tf26, on = id_cols, suffixes=suffixes, how = 'inner')

In [7]:
pd.set_option("display.precision", 7)

In [8]:
comparison = both[id_cols + [f'max_diff{s}' for s in suffixes]].copy()

In [9]:
comparison['diff-in-diff'] = (comparison['max_diff_27'] - comparison['max_diff_26'])

In [10]:
comparison.sort_values(['model_name','diff-in-diff'])

,model_name,model_weights,text,max_diff_27,max_diff_26,diff-in-diff
0,bert,None,Good evening.,0.0000005,0.0000005,0.0000000
1,bert,None,here is the sentence I want embeddings for.,0.0000010,0.0000010,0.0000000
2,bert,bert-base-uncased,Good evening.,0.0000015,0.0000015,0.0000000
3,bert,bert-base-uncased,here is the sentence I want embeddings for.,0.0000008,0.0000008,0.0000000
10,distilbert,None,Good evening.,0.0000003,0.0000003,0.0000000
11,distilbert,None,here is the sentence I want embeddings for.,0.0000004,0.0000004,0.0000000
5,gpt,None,here is the sentence I want embeddings for.,0.0039136,0.0000025,0.0039111
4,gpt,None,Good evening.,0.0042694,0.0000011,0.0042683
6,gpt2,None,Good evening.,0.0000006,0.0000006,0.0000000
7,gpt2,None,here is the sentence I want embeddings for.,0.0000010,0.0000010,0.0000000


In [11]:
comparison[comparison['diff-in-diff'] < 1e-7]['model_name'].unique()

array(['bert', 'gpt2', 'distilbert', 'roberta'], dtype=object)

In [12]:
comparison[comparison['diff-in-diff'] >= 1e-7]

,model_name,model_weights,text,max_diff_27,max_diff_26,diff-in-diff
4,gpt,None,Good evening.,0.0042694,0.0000011,0.0042683
5,gpt,None,here is the sentence I want embeddings for.,0.0039136,0.0000025,0.0039111
8,xlnet,None,Good evening.,0.0172069,0.0000060,0.0172009
9,xlnet,None,here is the sentence I want embeddings for.,0.0148172,0.0000118,0.0148054


In [13]:
comparison.to_csv('comparison.csv', index=False)

### Direct comparison of embeddings

In [14]:
direct_comparison = both.copy()
for type_ in ['sequence', 'sentence']:
    direct_comparison[f'max-diff-{type_}'] = direct_comparison.apply(lambda row: 
        max(abs(a-b) for a,b in zip(row[f'{type_}_actual_27'], row[f'{type_}_actual_26'])), axis=1
    )
direct_comparison[id_cols+[col for col in direct_comparison.columns if "max-dif" in col]]

,model_name,model_weights,text,max-diff-sequence,max-diff-sentence
0,bert,None,Good evening.,0.0000000,0.0000000
1,bert,None,here is the sentence I want embeddings for.,0.0000000,0.0000000
2,bert,bert-base-uncased,Good evening.,0.0000000,0.0000000
3,bert,bert-base-uncased,here is the sentence I want embeddings for.,0.0000000,0.0000000
4,gpt,None,Good evening.,0.0042703,0.0023828
5,gpt,None,here is the sentence I want embeddings for.,0.0039140,0.0009931
6,gpt2,None,Good evening.,0.0000000,0.0000000
7,gpt2,None,here is the sentence I want embeddings for.,0.0000000,0.0000000
8,xlnet,None,Good evening.,0.0032659,0.0172076
9,xlnet,None,here is the sentence I want embeddings for.,0.0091770,0.0148157
